In [1]:
from pymongo import MongoClient
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element, size_continuous_style, color_continuous_style

#### Conectamos mongo con pyhton

In [2]:
conn = MongoClient("localhost:27017")

In [3]:
# Para ver donde que databases tengo
conn.list_database_names()

['admin', 'config', 'ironhack', 'local']

In [4]:
# Elegimos Ironhack porque tendro tenemos la colleccion que queremos utilizar
db = conn.get_database("ironhack")

In [5]:
db.list_collection_names()

['companies', 'countries_small', 'books', 'restaurants']

In [6]:
# "Abrimos" nuestra collecion companies
collection = db.get_collection("companies")

### Empezamos a investigar dentro de nuestra colección
#### Hipótesis:

##### Datos que puedo encontrar en mi colección de de "Companies":
* Soy una gran emprendedora y quiero lazar mi nueva empresa
* Quiero ver cual sería la mejor ubicación para lanzar mi empresa
    - Estoy entre Madrid y Londres
* Además quiero estar rodeada tando de empresas teconológicas como de start-ups, me gusta mucho el ambiente joven e innovador y sin miedo a cambios y quiero encontrar ese entorno.

##### Datos que tengo que encontrar en Apis:
* Elegir 
* 3 
* condiciones del Readme


### 1. Busco aquellas empresas que estén ubicadas en Madrid y londres

In [7]:
collection.find_one({})

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [8]:
collection.find_one({}).keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

* Selección de tipo de empresas que quieres que se encuentres al rededor de la tuya.
* Nosotros lo que buscamos es:
    - Empresas ubicadas en Madrid o Londres
    - Empresas cuya categoria empresarial esté relacionada con el "ecommerce"
    - Y, empresas jovenes cuya año de fundación se ecuentre entre el 2005 y 2020. 
       ¡Spoiler! He buscado empresas de nueva creacion cuyo año de funcacion haya sido 2015 (ubicadas en Madrid o Londres y de catgortia "ecommer" y no hay.

In [9]:
collection.distinct('total_money_raised')

['$0',
 '$1.01M',
 '$1.02M',
 '$1.03M',
 '$1.04M',
 '$1.05M',
 '$1.06B',
 '$1.08M',
 '$1.09M',
 '$1.11M',
 '$1.13M',
 '$1.14M',
 '$1.15B',
 '$1.15M',
 '$1.16B',
 '$1.17M',
 '$1.19M',
 '$1.1B',
 '$1.1M',
 '$1.21M',
 '$1.23M',
 '$1.25M',
 '$1.27M',
 '$1.2B',
 '$1.2M',
 '$1.31M',
 '$1.32M',
 '$1.33M',
 '$1.35M',
 '$1.36M',
 '$1.3M',
 '$1.42M',
 '$1.44M',
 '$1.45M',
 '$1.46M',
 '$1.4M',
 '$1.51M',
 '$1.53M',
 '$1.55M',
 '$1.56M',
 '$1.57M',
 '$1.58M',
 '$1.5M',
 '$1.61M',
 '$1.62M',
 '$1.63M',
 '$1.65M',
 '$1.66M',
 '$1.67M',
 '$1.68M',
 '$1.69M',
 '$1.6M',
 '$1.71M',
 '$1.72M',
 '$1.73M',
 '$1.75M',
 '$1.77B',
 '$1.77M',
 '$1.7M',
 '$1.81M',
 '$1.82M',
 '$1.83M',
 '$1.84M',
 '$1.8M',
 '$1.92M',
 '$1.94M',
 '$1.95M',
 '$1.96M',
 '$1.97M',
 '$1.98M',
 '$1.99M',
 '$1.9M',
 '$10.1M',
 '$10.2M',
 '$10.3M',
 '$10.4M',
 '$10.5M',
 '$10.6M',
 '$10.7M',
 '$10.8M',
 '$10.9M',
 '$100M',
 '$100k',
 '$101M',
 '$101k',
 '$102M',
 '$103M',
 '$104M',
 '$105M',
 '$105k',
 '$106M',
 '$107M',
 '$108M',
 '$1

In [10]:
madrid_companies = collection.find({"offices.city": "Madrid", "category_code" : "ecommerce", "founded_year" : {"$gt": 2004}}, {"name": 1,"category_code":1, "founded_year":1, "offices":1, "total_money_raised": 1, "_id" : 0})

In [11]:
list(madrid_companies)

[{'name': 'Digital Assets Deployment',
  'category_code': 'ecommerce',
  'founded_year': 2006,
  'total_money_raised': '$0',
  'offices': [{'description': 'Corporate Headquarters',
    'address1': 'Doctor Castelo 10',
    'address2': 'Local 1',
    'zip_code': '28009',
    'city': 'Madrid',
    'state_code': None,
    'country_code': 'ESP',
    'latitude': 40.42046,
    'longitude': -3.678121}]},
 {'name': 'Daily Flat Rental',
  'category_code': 'ecommerce',
  'founded_year': 2008,
  'total_money_raised': '$0',
  'offices': [{'description': 'Central Office',
    'address1': 'Lavapies 26 1A',
    'address2': '',
    'zip_code': '28012',
    'city': 'Madrid',
    'state_code': None,
    'country_code': 'ESP',
    'latitude': 40.412323,
    'longitude': -3.703248}]},
 {'name': 'Daily Flat Rental',
  'category_code': 'ecommerce',
  'founded_year': 2008,
  'total_money_raised': '$0',
  'offices': [{'description': 'Central Office',
    'address1': 'Lavapies 26 1A',
    'address2': '',
    'z

***Empresas en Madrid***

In [12]:
madrid_companies = collection.find({"offices.city": "Madrid", "category_code" : "ecommerce", "founded_year" : {"$gt": 2004}}, {"name": 1,"category_code":1, "founded_year":1, "offices":1, "total_money_raised":1, "_id" : 0})

In [13]:
list(madrid_companies)

[{'name': 'Digital Assets Deployment',
  'category_code': 'ecommerce',
  'founded_year': 2006,
  'total_money_raised': '$0',
  'offices': [{'description': 'Corporate Headquarters',
    'address1': 'Doctor Castelo 10',
    'address2': 'Local 1',
    'zip_code': '28009',
    'city': 'Madrid',
    'state_code': None,
    'country_code': 'ESP',
    'latitude': 40.42046,
    'longitude': -3.678121}]},
 {'name': 'Daily Flat Rental',
  'category_code': 'ecommerce',
  'founded_year': 2008,
  'total_money_raised': '$0',
  'offices': [{'description': 'Central Office',
    'address1': 'Lavapies 26 1A',
    'address2': '',
    'zip_code': '28012',
    'city': 'Madrid',
    'state_code': None,
    'country_code': 'ESP',
    'latitude': 40.412323,
    'longitude': -3.703248}]},
 {'name': 'Daily Flat Rental',
  'category_code': 'ecommerce',
  'founded_year': 2008,
  'total_money_raised': '$0',
  'offices': [{'description': 'Central Office',
    'address1': 'Lavapies 26 1A',
    'address2': '',
    'z

In [14]:
df = pd.DataFrame(collection.find(
    {"$and": [
        {"offices.city": "Madrid"},
        {"category_code" : "ecommerce"},
        {"founded_year" : {"$gt": 2004}}
    ]},
    {"name": 1, 
     "category_code": 1, 
     "founded_year": 1, 
     "offices.city": 1, 
     "offices.latitude":1, 
     "total_money_raised":1,
     "number_of_employees":1,
     "offices.longitude":1, "_id": 0}
))

df

,name,category_code,number_of_employees,founded_year,total_money_raised,offices
0,Digital Assets Deployment,ecommerce,10.0,2006,$0,"[{'city': 'Madrid', 'latitude': 40.42046, 'lon..."
1,Daily Flat Rental,ecommerce,2.0,2008,$0,"[{'city': 'Madrid', 'latitude': 40.412323, 'lo..."
2,Daily Flat Rental,ecommerce,2.0,2008,$0,"[{'city': 'Madrid', 'latitude': 40.412323, 'lo..."
3,ticketea,ecommerce,NaN,2010,$5.73M,"[{'city': 'Madrid', 'latitude': 40.4455155, 'l..."
4,BuyVIP,ecommerce,NaN,2006,$20M,"[{'city': 'Madrid', 'latitude': 40.4167413, 'l..."


* Vamor a limpiar la columna de offcies y ponerlo todo mucho más visual para poder luego crear nuestros mapas de manera más sencilla y ordenada

In [15]:
df.offices

0    [{'city': 'Madrid', 'latitude': 40.42046, 'lon...
1    [{'city': 'Madrid', 'latitude': 40.412323, 'lo...
2    [{'city': 'Madrid', 'latitude': 40.412323, 'lo...
3    [{'city': 'Madrid', 'latitude': 40.4455155, 'l...
4    [{'city': 'Madrid', 'latitude': 40.4167413, 'l...
Name: offices, dtype: object

In [16]:
city = []
for i in list(df.offices):
    city.append((i[0]['city']))

In [17]:
df_city = pd.DataFrame(city)
df_city.columns = ['city']
df_city

,city
0,Madrid
1,Madrid
2,Madrid
3,Madrid
4,Madrid


In [18]:
latitude = []
for i in list(df.offices):
    latitude.append((i[0]['latitude']))

latitude

[40.42046, 40.412323, 40.412323, 40.4455155, 40.4167413]

In [19]:
df_latitude = pd.DataFrame(latitude)
df_latitude.columns = ['latitude']
df_latitude

,latitude
0,40.420460
1,40.412323
2,40.412323
3,40.445515
4,40.416741


In [20]:
longitude = []
for i in list(df.offices):
    longitude.append((i[0]['longitude']))

longitude

[-3.678121, -3.703248, -3.703248, -3.7061764, -3.7032498]

In [21]:
df_longitude = pd.DataFrame(longitude)
df_longitude.columns = ['longitude']
df_longitude

,longitude
0,-3.678121
1,-3.703248
2,-3.703248
3,-3.706176
4,-3.703250


In [61]:
# quitamos los valores NAN porque no quedan nada profesionales
df_Madrid_nulls = pd.DataFrame(df[['number_of_employees']])
                                           
df_Madrid_nulls['fill_0_employees'] = df_Madrid_nulls['number_of_employees'].fillna(0) 
df_Madrid_nulls[['number_of_employees', 'fill_0_employees']]

,number_of_employees,fill_0_employees
0,10.0,10.0
1,2.0,2.0
2,2.0,2.0
3,NaN,0.0
4,NaN,0.0


In [62]:
df_Madrid = pd.concat([df, df_city, df_latitude, df_longitude, df_Madrid_nulls], axis=1)
df_Madrid

,name,category_code,number_of_employees,founded_year,total_money_raised,offices,city,latitude,longitude,number_of_employees,fill_0_employees
0,Digital Assets Deployment,ecommerce,10.0,2006,$0,"[{'city': 'Madrid', 'latitude': 40.42046, 'lon...",Madrid,40.420460,-3.678121,10.0,10.0
1,Daily Flat Rental,ecommerce,2.0,2008,$0,"[{'city': 'Madrid', 'latitude': 40.412323, 'lo...",Madrid,40.412323,-3.703248,2.0,2.0
2,Daily Flat Rental,ecommerce,2.0,2008,$0,"[{'city': 'Madrid', 'latitude': 40.412323, 'lo...",Madrid,40.412323,-3.703248,2.0,2.0
3,ticketea,ecommerce,NaN,2010,$5.73M,"[{'city': 'Madrid', 'latitude': 40.4455155, 'l...",Madrid,40.445515,-3.706176,NaN,0.0
4,BuyVIP,ecommerce,NaN,2006,$20M,"[{'city': 'Madrid', 'latitude': 40.4167413, 'l...",Madrid,40.416741,-3.703250,NaN,0.0


In [63]:
# Eliminamos nuestra columna de offices y de employees sucia (ya que ya hemos sacado los datos que queriamos y los hemps pues de manera ordenada)
df_Madrid_clean= df_Madrid.drop(df_Madrid.columns[[2, 5]], axis = 'columns')
df_Madrid_clean

,name,category_code,founded_year,total_money_raised,city,latitude,longitude,fill_0_employees
0,Digital Assets Deployment,ecommerce,2006,$0,Madrid,40.420460,-3.678121,10.0
1,Daily Flat Rental,ecommerce,2008,$0,Madrid,40.412323,-3.703248,2.0
2,Daily Flat Rental,ecommerce,2008,$0,Madrid,40.412323,-3.703248,2.0
3,ticketea,ecommerce,2010,$5.73M,Madrid,40.445515,-3.706176,0.0
4,BuyVIP,ecommerce,2006,$20M,Madrid,40.416741,-3.703250,0.0


In [64]:
# Quitar el signo del dolar y de millones

df_Madrid_clean["total_money_raised"] = df_Madrid_clean["total_money_raised"].str.replace('$', '')
df_Madrid_clean["total_money_raised"] = df_Madrid_clean["total_money_raised"].str.replace('M', '')
df_Madrid_clean["total_money_raised"]

<ipython-input-64-a5539623414f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Madrid_clean["total_money_raised"] = df_Madrid_clean["total_money_raised"].str.replace('$', '')
<ipython-input-64-a5539623414f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Madrid_clean["total_money_raised"] = df_Madrid_clean["total_money_raised"].str.replace('M', '')


0       0
1       0
2       0
3    5.73
4      20
Name: total_money_raised, dtype: object

In [65]:
#Quitamos el signo del dólar de la comuna "total_money_raised" porque al hacer el mapa necesitamos una variable numérica NO categórica
df_Madrid_clean["total_money_raised"] = df_Madrid_clean["total_money_raised"].astype(float)
df_Madrid_clean["total_money_raised"]

<ipython-input-65-c03fd3f3e2e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Madrid_clean["total_money_raised"] = df_Madrid_clean["total_money_raised"].astype(float)


0     0.00
1     0.00
2     0.00
3     5.73
4    20.00
Name: total_money_raised, dtype: float64

In [128]:
gdf = gpd.GeoDataFrame(df_Madrid_clean, geometry=gpd.points_from_xy(df_Madrid.longitude, df_Madrid.latitude))
print(f'Tipo: {type(gdf)}')
gdf

Tipo: <class 'geopandas.geodataframe.GeoDataFrame'>


,name,category_code,founded_year,total_money_raised,city,latitude,longitude,fill_0_employees,geometry
0,Digital Assets Deployment,ecommerce,2006,0.00,Madrid,40.420460,-3.678121,10.0,POINT (-3.67812 40.42046)
1,Daily Flat Rental,ecommerce,2008,0.00,Madrid,40.412323,-3.703248,2.0,POINT (-3.70325 40.41232)
2,Daily Flat Rental,ecommerce,2008,0.00,Madrid,40.412323,-3.703248,2.0,POINT (-3.70325 40.41232)
3,ticketea,ecommerce,2010,5.73,Madrid,40.445515,-3.706176,0.0,POINT (-3.70618 40.44552)
4,BuyVIP,ecommerce,2006,20.00,Madrid,40.416741,-3.703250,0.0,POINT (-3.70325 40.41674)


In [129]:
df_Madrid_clean

,name,category_code,founded_year,total_money_raised,city,latitude,longitude,fill_0_employees,geometry
0,Digital Assets Deployment,ecommerce,2006,0.00,Madrid,40.420460,-3.678121,10.0,POINT (-3.67812 40.42046)
1,Daily Flat Rental,ecommerce,2008,0.00,Madrid,40.412323,-3.703248,2.0,POINT (-3.70325 40.41232)
2,Daily Flat Rental,ecommerce,2008,0.00,Madrid,40.412323,-3.703248,2.0,POINT (-3.70325 40.41232)
3,ticketea,ecommerce,2010,5.73,Madrid,40.445515,-3.706176,0.0,POINT (-3.70618 40.44552)
4,BuyVIP,ecommerce,2006,20.00,Madrid,40.416741,-3.703250,0.0,POINT (-3.70325 40.41674)


In [130]:
Map(Layer(gdf, popup_hover=[popup_element("name")]))

In [131]:
Map(Layer(gdf, size_continuous_style('total_money_raised', size_range=[10,40]), title='Total money raised ($/Units of millon)'))

In [132]:
Map([
  Layer(gdf, color_continuous_style('fill_0_employees'), title='Number of employees'),
  Layer(gdf, size_continuous_style('total_money_raised', size_range=[20,40]), title='Total money raised ($/units of million)') 
])

***Empresas en Londres***

In [71]:
london_companies = collection.find({"offices.city": "London", "category_code" : "ecommerce", "founded_year" : {"$gt": 2004}}, {"name": 1, "category_code":1,"founded_year":1, "_id" : 0})

In [72]:
list(london_companies)

[{'name': 'Twenga', 'category_code': 'ecommerce', 'founded_year': 2006},
 {'name': 'Seatwave', 'category_code': 'ecommerce', 'founded_year': 2006},
 {'name': 'ProcServe', 'category_code': 'ecommerce', 'founded_year': 2006},
 {'name': 'Voices', 'category_code': 'ecommerce', 'founded_year': 2005},
 {'name': 'Veedow', 'category_code': 'ecommerce', 'founded_year': 2007},
 {'name': 'Spectackler', 'category_code': 'ecommerce', 'founded_year': 2008},
 {'name': 'Shutl', 'category_code': 'ecommerce', 'founded_year': 2009},
 {'name': 'Oaven', 'category_code': 'ecommerce', 'founded_year': 2006},
 {'name': 'Aroxo', 'category_code': 'ecommerce', 'founded_year': 2006},
 {'name': 'Autoquake', 'category_code': 'ecommerce', 'founded_year': 2005},
 {'name': 'Schway', 'category_code': 'ecommerce', 'founded_year': 2007},
 {'name': 'emarket', 'category_code': 'ecommerce', 'founded_year': 2008}]

In [78]:
df_London = pd.DataFrame(collection.find(
    {"$and": [
        {"offices.city": "London"},
        {"category_code" : "ecommerce"},
        {"founded_year" : {"$gt": 2004}}
    ]},
    {"name": 1, 
     "category_code": 1, 
     "founded_year": 1,
     "number_of_employees": 1,
     "total_money_raised" : 1,
     "offices.city":1,
     "offices.longitude": 1,
     "offices.latitude": 1, 
     "_id": 0}
))

df_London

,name,category_code,number_of_employees,founded_year,total_money_raised,offices
0,Twenga,ecommerce,NaN,2006,€2.6M,"[{'city': 'Paris', 'latitude': 48.856667, 'lon..."
1,Seatwave,ecommerce,NaN,2006,$53M,"[{'city': 'London', 'latitude': 51.500152, 'lo..."
2,ProcServe,ecommerce,55.0,2006,$0,"[{'city': 'London', 'latitude': 51.494067, 'lo..."
3,Voices,ecommerce,23.0,2005,$0,"[{'city': 'London', 'latitude': 43.014006, 'lo..."
4,Veedow,ecommerce,4.0,2007,$0,"[{'city': 'London', 'latitude': None, 'longitu..."
5,Spectackler,ecommerce,3.0,2008,$0,"[{'city': 'London', 'latitude': None, 'longitu..."
6,Shutl,ecommerce,10.0,2009,£7.66M,"[{'city': 'London', 'latitude': 51.5196135, 'l..."
7,Oaven,ecommerce,4.0,2006,$0,"[{'city': 'London', 'latitude': None, 'longitu..."
8,Aroxo,ecommerce,10.0,2006,$0,"[{'city': 'London', 'latitude': None, 'longitu..."
9,Autoquake,ecommerce,100.0,2005,$39.9M,"[{'city': 'London', 'latitude': 51.543337, 'lo..."


In [60]:
# quitamos los valores NAN porque no quedan nada profesionales
df_London_nulls = pd.DataFrame(df_London[['number_of_employees']])
                                           
df_London_nulls['fill_0_employees'] = df_London_nulls['number_of_employees'].fillna(0) 
df_London_nulls[['number_of_employees', 'fill_0_employees']]

,number_of_employees,fill_0_employees
0,NaN,0.0
1,NaN,0.0
2,55.0,55.0
3,23.0,23.0
4,4.0,4.0
5,3.0,3.0
6,10.0,10.0
7,4.0,4.0
8,10.0,10.0
9,100.0,100.0


In [77]:
df_London.offices[1]

[{'city': 'London'}]

In [54]:
city_L = []
for i in list(df_London.offices):
    city_L.append((i[0]['city']))

In [51]:
df_city_L = pd.DataFrame(city_L)
df_city_L.columns = ['city']
df_city_L

,city
0,Paris
1,London
2,London
3,London
4,London
5,London
6,London
7,London
8,London
9,London


In [79]:
longitude_L = []
for i in list(df_London.offices):
    longitude_L.append((i[0]['longitude']))

In [80]:
df_longitude_L = pd.DataFrame(longitude_L)
df_longitude_L.columns = ['longitude']
df_longitude_L

,longitude
0,2.350987
1,-0.126236
2,-0.146665
3,-81.280364
4,NaN
5,NaN
6,-0.102027
7,NaN
8,NaN
9,-0.172364


In [81]:
latitude_L = []
for i in list(df_London.offices):
    latitude_L.append((i[0]['latitude']))

In [82]:
df_latitude_L = pd.DataFrame(latitude_L)
df_latitude_L.columns = ['latitude']
df_latitude_L

,latitude
0,48.856667
1,51.500152
2,51.494067
3,43.014006
4,NaN
5,NaN
6,51.519613
7,NaN
8,NaN
9,51.543337


In [84]:
df_London_clean = pd.concat([df_London, df_city_L, df_latitude_L, df_longitude_L, df_London_nulls], axis=1)
df_London_clean

,name,category_code,number_of_employees,founded_year,total_money_raised,offices,city,latitude,longitude,number_of_employees,fill_0_employees
0,Twenga,ecommerce,NaN,2006,€2.6M,"[{'city': 'Paris', 'latitude': 48.856667, 'lon...",Paris,48.856667,2.350987,NaN,0.0
1,Seatwave,ecommerce,NaN,2006,$53M,"[{'city': 'London', 'latitude': 51.500152, 'lo...",London,51.500152,-0.126236,NaN,0.0
2,ProcServe,ecommerce,55.0,2006,$0,"[{'city': 'London', 'latitude': 51.494067, 'lo...",London,51.494067,-0.146665,55.0,55.0
3,Voices,ecommerce,23.0,2005,$0,"[{'city': 'London', 'latitude': 43.014006, 'lo...",London,43.014006,-81.280364,23.0,23.0
4,Veedow,ecommerce,4.0,2007,$0,"[{'city': 'London', 'latitude': None, 'longitu...",London,NaN,NaN,4.0,4.0
5,Spectackler,ecommerce,3.0,2008,$0,"[{'city': 'London', 'latitude': None, 'longitu...",London,NaN,NaN,3.0,3.0
6,Shutl,ecommerce,10.0,2009,£7.66M,"[{'city': 'London', 'latitude': 51.5196135, 'l...",London,51.519613,-0.102027,10.0,10.0
7,Oaven,ecommerce,4.0,2006,$0,"[{'city': 'London', 'latitude': None, 'longitu...",London,NaN,NaN,4.0,4.0
8,Aroxo,ecommerce,10.0,2006,$0,"[{'city': 'London', 'latitude': None, 'longitu...",London,NaN,NaN,10.0,10.0
9,Autoquake,ecommerce,100.0,2005,$39.9M,"[{'city': 'London', 'latitude': 51.543337, 'lo...",London,51.543337,-0.172364,100.0,100.0


In [86]:
df_London_super_clean= df_London_clean.drop(df_London_clean.columns[[2, 5]], axis = 'columns')
df_London_super_clean

,name,category_code,founded_year,total_money_raised,city,latitude,longitude,fill_0_employees
0,Twenga,ecommerce,2006,€2.6M,Paris,48.856667,2.350987,0.0
1,Seatwave,ecommerce,2006,$53M,London,51.500152,-0.126236,0.0
2,ProcServe,ecommerce,2006,$0,London,51.494067,-0.146665,55.0
3,Voices,ecommerce,2005,$0,London,43.014006,-81.280364,23.0
4,Veedow,ecommerce,2007,$0,London,NaN,NaN,4.0
5,Spectackler,ecommerce,2008,$0,London,NaN,NaN,3.0
6,Shutl,ecommerce,2009,£7.66M,London,51.519613,-0.102027,10.0
7,Oaven,ecommerce,2006,$0,London,NaN,NaN,4.0
8,Aroxo,ecommerce,2006,$0,London,NaN,NaN,10.0
9,Autoquake,ecommerce,2005,$39.9M,London,51.543337,-0.172364,100.0


In [107]:
# Quitar el signo del dolar y de millones

df_London_super_clean["total_money_raised"] = df_London_super_clean["total_money_raised"].str.replace('$', '')
df_London_super_clean["total_money_raised"] = df_London_super_clean["total_money_raised"].str.replace('£', '')
df_London_super_clean["total_money_raised"] = df_London_super_clean["total_money_raised"].str.replace('€', '')
df_London_super_clean["total_money_raised"] = df_London_super_clean["total_money_raised"].str.replace('M', '')
df_London_super_clean["total_money_raised"]

<ipython-input-107-80df8a8f41ef>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_London_super_clean["total_money_raised"] = df_London_super_clean["total_money_raised"].str.replace('$', '')
<ipython-input-107-80df8a8f41ef>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_London_super_clean["total_money_raised"] = df_London_super_clean["total_money_raised"].str.replace('£', '')
<ipython-input-107-80df8a8f41ef>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

0      2.6
1       53
2        0
3        0
4        0
5        0
6     7.66
7        0
8        0
9     39.9
10       0
11       0
Name: total_money_raised, dtype: object

In [108]:
#Quitamos el signo del dólar de la comuna "total_money_raised" porque al hacer el mapa necesitamos una variable numérica NO categórica
df_London_super_clean["total_money_raised"] = df_London_super_clean["total_money_raised"].astype(float)
df_London_super_clean["total_money_raised"]

<ipython-input-108-ff4a906adf8b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_London_super_clean["total_money_raised"] = df_London_super_clean["total_money_raised"].astype(float)


0      2.60
1     53.00
2      0.00
3      0.00
4      0.00
5      0.00
6      7.66
7      0.00
8      0.00
9     39.90
10     0.00
11     0.00
Name: total_money_raised, dtype: float64

In [109]:
# Como este proyecto se basa en ubicaniones y por tanto en coordenadas tengo que eliminar las empresas que NO tenga coordenadas
#df_London_super_clean= df_London_clean.drop(df_London_clean[[4, 5, 7, 8, 11]], axis = 0)
#df_London_super_clean
df_London_limpio = df_London_super_clean.drop([0, 4, 5, 7, 8, 11],axis=0)

In [110]:
gdf_London = gpd.GeoDataFrame(df_London_limpio, geometry=gpd.points_from_xy(df_London_limpio.longitude, df_London_limpio.latitude))
print(f'Tipo: {type(gdf)}')
gdf_London

Tipo: <class 'geopandas.geodataframe.GeoDataFrame'>


,name,category_code,founded_year,total_money_raised,city,latitude,longitude,fill_0_employees,geometry
1,Seatwave,ecommerce,2006,53.00,London,51.500152,-0.126236,0.0,POINT (-0.12624 51.50015)
2,ProcServe,ecommerce,2006,0.00,London,51.494067,-0.146665,55.0,POINT (-0.14666 51.49407)
3,Voices,ecommerce,2005,0.00,London,43.014006,-81.280364,23.0,POINT (-81.28036 43.01401)
6,Shutl,ecommerce,2009,7.66,London,51.519613,-0.102027,10.0,POINT (-0.10203 51.51961)
9,Autoquake,ecommerce,2005,39.90,London,51.543337,-0.172364,100.0,POINT (-0.17236 51.54334)
10,Schway,ecommerce,2007,0.00,London,51.519816,-0.136162,5.0,POINT (-0.13616 51.51982)


In [133]:
Map(Layer(gdf_London, popup_hover=[popup_element("name")]))

In [111]:
Map(Layer(gdf_London, size_continuous_style('total_money_raised', size_range=[10,40]), title='Total money raised ($/Units of millon)'))

In [125]:
Map([
  Layer(gdf_London, color_continuous_style('fill_0_employees'), title='Number of employees'),
  Layer(gdf_London, size_continuous_style('total_money_raised', size_range=[20,50]), title='Total money raised ($/units of million)') 
])